In [1]:
# pip install nfl_data_py

In [2]:
# # Install a set of mutually compatible versions
# %pip install -qU \
#   "pandas==2.2.2" "numpy==2.0.2" "xarray==2024.06.0" \
#   "scikit-learn==1.6.1" "pyarrow==19.*" \
#   lightgbm rapidfuzz pandas-gbq google-cloud-bigquery

# # Programmatically restart the kernel so the new versions actually load
# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

In [3]:
import nfl_data_py as nfl
import pandas as pd, numpy as np, xarray as xr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, mean_absolute_error
from lightgbm import LGBMClassifier, LGBMRegressor

from google.cloud import bigquery
import pandas_gbq

print("pandas:", pd.__version__, "| numpy:", np.__version__, "| xarray:", xr.__version__)


pandas: 2.2.2 | numpy: 2.0.2 | xarray: 2024.6.0


In [4]:
# list(nfl.see_pbp_cols())

In [5]:
years = list(range(2019, 2024))  # expand to 1999–2024 later if you want
base_cols = [
    "play_id","game_id","season","season_type","week","game_date",
    "home_team","away_team","posteam","defteam","posteam_type",
    "qtr","down","ydstogo","yardline_100","goal_to_go",
    "time","game_seconds_remaining","score_differential",
    "shotgun","no_huddle","qb_dropback","xpass",
    "play_type","yards_gained","epa","success",
    "home_score","away_score","home_wp","away_wp","home_wp_post","away_wp_post",
    "vegas_wp","vegas_home_wp","spread_line","total_line","result"
]

pbp = nfl.import_pbp_data(
    years=years,
    columns=base_cols,
    downcast=True,   # saves memory
    cache=False,     # set True after first run if you want faster reloads
    alt_path=None    # set if you cached elsewhere via nfl.cache_pbp()
)
pbp.shape, pbp.head(3)


2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


((243986, 58),
    play_id          game_id season_type  week   game_date home_team away_team  \
 0      1.0  2019_01_ATL_MIN         REG     1  2019-09-08       MIN       ATL   
 1     36.0  2019_01_ATL_MIN         REG     1  2019-09-08       MIN       ATL   
 2     51.0  2019_01_ATL_MIN         REG     1  2019-09-08       MIN       ATL   
 
   posteam defteam posteam_type  ...  \
 0    None    None         None  ...   
 1     ATL     MIN         away  ...   
 2     ATL     MIN         away  ...   
 
                                      offense_players  \
 0                                                      
 1  00-0035150;00-0033336;00-0032918;00-0032422;00...   
 2  00-0035235;00-0028042;00-0031279;00-0026997;00...   
 
                                      defense_players  n_offense  n_defense  \
 0                                                           0.0        0.0   
 1  00-0034411;00-0031570;00-0028042;00-0030968;00...       11.0       11.0   
 2  00-0031256;00-0027885;

In [6]:
# Create a simplified target
def simplify_play_type(pt):
    if pt in ["pass", "run"]:
        return pt
    if pt in ["punt", "field_goal","qb_kneel","qb_spike"]:
        return pt
    return "other"

pbp["play_type_simple"] = pbp["play_type"].fillna("other").map(simplify_play_type)

# Keep pre-snap features only (avoid yards_gained, epa, *_post, etc. as predictors)
feat_cols = [
    "season","week","qtr","down","ydstogo","yardline_100","goal_to_go",
    "game_seconds_remaining","score_differential",
    "shotgun","no_huddle","qb_dropback","xpass",
    "posteam","defteam","home_team","away_team","posteam_type"
]
X = pbp[feat_cols].copy()
y = pbp["play_type_simple"]

# Train/val split by SEASON to reduce leakage (train prior seasons, test last season)
train = pbp["season"] < max(pbp["season"])
X_train, X_test = X[train], X[~train]
y_train, y_test = y[train], y[~train]

num_cols = ["season","week","qtr","down","ydstogo","yardline_100","goal_to_go",
            "game_seconds_remaining","score_differential","shotgun","no_huddle","qb_dropback","xpass"]
cat_cols = ["posteam","defteam","home_team","away_team","posteam_type"]

pre = ColumnTransformer([
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_cols)
], remainder="passthrough")

clf = Pipeline([
    ("pre", pre),
    ("gbm", LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=-1, n_jobs=-1))
])
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print("Play-type accuracy:", round(acc, 3))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011841 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1060
[LightGBM] [Info] Number of data points in the train set: 194321, number of used features: 146
[LightGBM] [Info] Start training from score -3.831099
[LightGBM] [Info] Start training from score -1.570137
[LightGBM] [Info] Start training from score -0.869007
[LightGBM] [Info] Start training from score -3.104122
[LightGBM] [Info] Start training from score -4.727769
[LightGBM] [Info] Start training from score -6.470156
[LightGBM] [Info] Start training from score -1.218509
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Play-type accuracy: 0.937


In [7]:
y_reg = pbp.loc[train, "epa"].fillna(0.0)
X_reg = X[train]
X_reg_test = X[~train]
y_reg_test = pbp.loc[~train, "epa"].fillna(0.0)

reg = Pipeline([
    ("pre", pre),
    ("gbm", LGBMRegressor(n_estimators=500, learning_rate=0.05, num_leaves=63, n_jobs=-1))
])
reg.fit(X_reg, y_reg)
pred_reg = reg.predict(X_reg_test)
print("Play-level EPA MAE:", round(mean_absolute_error(y_reg_test, pred_reg), 4))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1060
[LightGBM] [Info] Number of data points in the train set: 194321, number of used features: 146
[LightGBM] [Info] Start training from score -0.004828


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Play-level EPA MAE: 0.7998


In [8]:
# Game-level frame
gcols = ["game_id","season","week","home_team","away_team","home_score","away_score",
         "spread_line","total_line","vegas_home_wp"]
g = pbp[gcols].drop_duplicates("game_id").copy()

g["home_win"] = (g["home_score"] > g["away_score"]).astype(int)

# Add simple rolling team strength (last 3 games offensive EPA)
team_off_epa = (
    pbp.groupby(["posteam","game_id"], as_index=False)["epa"].mean()
      .rename(columns={"posteam":"team","epa":"team_mean_epa"})
)

# Merge home/away rolling EPA (lagged)
def make_rolling(ts_df, team_col="team", value_col="team_mean_epa"):
    ts_df = ts_df.merge(
        pbp[["game_id","season","week"]].drop_duplicates(),
        on="game_id", how="left"
    ).sort_values([team_col,"season","week"])
    ts_df["team_off_epa_rolling3"] = (
        ts_df.groupby(team_col)[value_col].transform(lambda s: s.shift().rolling(3, min_periods=1).mean())
    )
    return ts_df[["game_id", team_col, "team_off_epa_rolling3"]]

roll = make_rolling(team_off_epa)

home_strength = roll.rename(columns={"team":"home_team","team_off_epa_rolling3":"home_off_epa_r3"})
away_strength = roll.rename(columns={"team":"away_team","team_off_epa_rolling3":"away_off_epa_r3"})

g = g.merge(home_strength, on=["game_id","home_team"], how="left") \
     .merge(away_strength, on=["game_id","away_team"], how="left")

feat = ["season","week","spread_line","total_line","vegas_home_wp","home_off_epa_r3","away_off_epa_r3",
        "home_team","away_team"]
Xg = g[feat].copy()
yg = g["home_win"]

train_g = g["season"] < max(g["season"])
Xg_tr, Xg_te = Xg[train_g], Xg[~train_g]
yg_tr, yg_te = yg[train_g], yg[~train_g]

pre_g = ColumnTransformer([
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True), ["home_team","away_team"])
], remainder="passthrough")

game_clf = Pipeline([
    ("pre", pre_g),
    ("gbm", LGBMClassifier(n_estimators=600, learning_rate=0.03, n_jobs=-1))
])
game_clf.fit(Xg_tr, yg_tr)
proba = game_clf.predict_proba(Xg_te)[:,1]
print("Game AUC:", round(roc_auc_score(yg_te, proba), 3))
print("Game accuracy (0.5 cut):", round(accuracy_score(yg_te, (proba>=0.5).astype(int)), 3))


[LightGBM] [Info] Number of positive: 580, number of negative: 525
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 859
[LightGBM] [Info] Number of data points in the train set: 1105, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.524887 -> initscore=0.099630
[LightGBM] [Info] Start training from score 0.099630
Game AUC: 0.598
Game accuracy (0.5 cut): 0.568


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [9]:
# Attach probabilities to test-season games and simulate
test_season = g["season"].max()
g_test = g[g["season"]==test_season].copy()

Xg_test_only = Xg_te.copy()
g_test["home_win_prob"] = proba

# Simulate many seasons (assumes schedule completed; works midseason too)
N = 2000
teams = sorted(set(g_test["home_team"]).union(g_test["away_team"]))
wins = pd.DataFrame(0, index=teams, columns=range(N), dtype=int)

rng = np.random.default_rng(0)
for sim in range(N):
    # Simulate each game outcome
    r = rng.random(len(g_test))
    home_wins = (r < g_test["home_win_prob"]).astype(int)
    # Assign wins
    for (_, row), hw in zip(g_test.iterrows(), home_wins):
        h, a = row["home_team"], row["away_team"]
        if hw == 1:
            wins.loc[h, sim] += 1
        else:
            wins.loc[a, sim] += 1

season_summary = pd.DataFrame({
    "team": wins.index,
    "avg_wins": wins.mean(axis=1),
    "p10_wins": wins.quantile(0.10, axis=1),
    "p50_wins": wins.quantile(0.50, axis=1),
    "p90_wins": wins.quantile(0.90, axis=1),
}).sort_values("avg_wins", ascending=False)
season_summary.head(10)


,team,avg_wins,p10_wins,p50_wins,p90_wins
KC,KC,17.6480,16.0,18.0,19.0
BUF,BUF,14.6610,13.0,15.0,16.0
SF,SF,14.2700,12.0,14.0,16.0
MIA,MIA,12.2915,11.0,12.0,14.0
BAL,BAL,11.9215,10.0,12.0,14.0
DAL,DAL,11.6755,10.0,12.0,13.0
DET,DET,11.6505,10.0,12.0,14.0
PHI,PHI,11.2770,9.0,11.0,13.0
NO,NO,10.9795,9.0,11.0,13.0
GB,GB,10.6970,9.0,11.0,13.0


In [10]:
project_id = "daring-emitter-469314-f5"
dataset = "nfl"
table = "pbp_2019_2023"

# create dataset once (idempotent)
bq = bigquery.Client(project=project_id)
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset}")
try:
    bq.get_dataset(dataset_ref)
except Exception:
    bq.create_dataset(dataset_ref)

# write data
pandas_gbq.to_gbq(pbp, f"{dataset}.{table}", project_id=project_id, if_exists="replace")


100%|██████████| 1/1 [00:00<00:00, 7516.67it/s]
